In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

In [22]:
def get_log_time():
    return datetime.now().strftime('%H:%M:%S %d/%m/%Y')

In [24]:
# ruta de carpera donde estan las llaves
print(get_log_time(), '> Estableciento conexion a base de datos...')
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud
print(now_hour, '> Base de datos conectado...!!!')

14:53:09 22/05/2023 > Estableciento conexion a base de datos...


14:50:52 22/05/2023 > Base de datos conectado...!!!


In [25]:
ruta

'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [30]:
# obtener fecha actual
fecha = datetime.now().strftime('%Y-%m-%d')
print(get_log_time() , '> Buscando nuevos registros...')
query_dia = {'query': 'select id, canhero from dataset_315614 where date(fecha_de_registro) = \'{fech}\' order by id'.format(fech=fecha)}
project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
select_dia = amigocloud.get(project, query_dia)
data_dia = select_dia['data']
print(get_log_time(), '> Nuevos registros encontrados:', len(data_dia))

14:55:58 22/05/2023 > Buscando nuevos registros...
14:55:58 22/05/2023 > Nuevos registros encontrados: 2


In [31]:
data_dia

[{'id': 105, 'canhero': '3469 / ENRIQUEZ PAICHUCAMA SIMON '},
 {'id': 106, 'canhero': '3469 / ENRIQUEZ PAICHUCAMA SIMON '}]

In [32]:
for id in data_dia:
    print('ID inspeccion: ', id['id'])
    id_insp = id['id']
    # DATOS DE LAS INPECCION
    query = {'query': 'select * from dataset_315614 where id={id_insp}'.format(id_insp=id_insp)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]

    # convertir fecha a formato dd/mm/aaaa
    data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')

    # los campos booleanos tiene formacto 'num_num', esto no es admitido para convertir a objeto
    # se modifica el dict para agregar 'k_' al inicio de todas llaves
    data = {'k_'+key : value for (key, value) in data.items()}
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    
    # FOTOS
    query_fotos = {'query': 'select gal.s3_filename, fot.obs from dataset_315689 fot\
                                inner join gallery_48937 gal on fot.amigo_id=gal.source_amigo_id\
                                where fot.insp_ref_fot=\'{id}\''.format(id=insp.k_amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    fotos = amigocloud.get(project, query_fotos)['data']
    
    # CREAR REPORTE
    doc = DocxTemplate(ruta + '/templates/tpl_insp_cosechadoras_preventivo.docx')
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({
            'foto': docxtpl.InlineImage(doc, image_descriptor= ruta + '/fotos/' + foto['s3_filename'], width=Mm(50)),
            'obs': foto['obs']
        })
    context = {'insp':insp, 'fotos':lista_fotos_inline}
    #context = {'insp':insp}
    doc.render(context)
    # formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
    cod_nom = insp.k_canhero.split(' / ')
    file_name = cod_nom[0] + '_ECP_' + insp.k_fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.k_id)
    #doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
    doc.save(ruta + r'/informes/_' + file_name + '.docx')
    print(file_name)
print('-------------------FIN')

ID inspeccion:  105
3469_ECP_22-05-2023_ENRIQUEZ PAICHUCAMA SIMON _105
ID inspeccion:  106
3469_ECP_22-05-2023_ENRIQUEZ PAICHUCAMA SIMON _106
-------------------FIN


In [21]:
lista_fotos_inline

[{'foto': <docxtpl.InlineImage at 0x2170850e808>, 'obs': 'Máquina lista '}]

## Convertir a PDF

In [10]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 18 INFORMES
18 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/10697_ECP_28-03-2023_PAZ REA JAIME EDUARDO _4.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/2446_ECP_16-05-2023_CORDOVA SEJAS ALCIDES _101.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3400_ECP_30-03-2023_ESPINOZA AGUILAR JAIME OMAR _11.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3469_ECP_22-05-2023_ENRIQUEZ PAICHUCAMA SIMON _106.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/4171_ECP_19-05-2023_FRITZER CUELLAR SAMIR EDUARDO _104.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/515_ECP_29-03-2023_AGUILERA OLGA RIVERO VDA. DE _7.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/556_ECP_16-05-2023_AYALA ESPINOSA WALDO SELUB_100.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/556_ECP_30-03-2023_AYALA ESPINO

In [11]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\UTEA\Downloads\\' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/10697_ECP_28-03-2023_PAZ REA JAIME EDUARDO _4.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/10697_ECP_28-03-2023_PAZ REA JAIME EDUARDO _4.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/10697_ECP_28-03-2023_PAZ REA JAIME EDUARDO _4.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 2 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2446_ECP_16-05-2023_CORDOVA SEJAS ALCIDES _101.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2446_ECP_16-05-2023_CORDOVA SEJAS ALCIDES _101.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/2446_ECP_16-05-2023_CORDOVA SEJAS ALCIDES _101.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 3 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3400_ECP_30-03-2023_ESPINOZA AGUILAR JAIME OMAR _11.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3400_ECP_30-03-2023_ESPINOZA AGUILAR JAIME OMAR _11.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/3400_ECP_30-03-2023_ESPINOZA AGUILAR JAIME OMAR _11.pdf


...

CONVIRTIENDO... INFORME 17 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/8009_ECP_29-03-2023_ESPINOZA AGUILAR DAVID_9.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/8009_ECP_29-03-2023_ESPINOZA AGUILAR DAVID_9.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/8009_ECP_29-03-2023_ESPINOZA AGUILAR DAVID_9.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 18 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/805_ECP_31-03-2023_AGROP. TRES HNOS DEL ORIENTE_17.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/805_ECP_31-03-2023_AGROP. TRES HNOS DEL ORIENTE_17.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/805_ECP_31-03-2023_AGROP. TRES HNOS DEL ORIENTE_17.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END


## SUBIR A DRIVE

In [12]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [14]:
contenido = os.listdir(ruta + r'//pdf')
dic_files = {}
for file in contenido:
    split = file.split('_')
    cod = split[-1].split('.')
    dic_files[int(cod[0])] = file
print(len(dic_files), 'informes')
dic_files

18 informes


{4: '10697_ECP_28-03-2023_PAZ REA JAIME EDUARDO _4.pdf',
 101: '2446_ECP_16-05-2023_CORDOVA SEJAS ALCIDES _101.pdf',
 11: '3400_ECP_30-03-2023_ESPINOZA AGUILAR JAIME OMAR _11.pdf',
 106: '3469_ECP_22-05-2023_ENRIQUEZ PAICHUCAMA SIMON _106.pdf',
 104: '4171_ECP_19-05-2023_FRITZER CUELLAR SAMIR EDUARDO _104.pdf',
 7: '515_ECP_29-03-2023_AGUILERA OLGA RIVERO VDA. DE _7.pdf',
 100: '556_ECP_16-05-2023_AYALA ESPINOSA WALDO SELUB_100.pdf',
 12: '556_ECP_30-03-2023_AYALA ESPINOSA WALDO SELUB_12.pdf',
 10: '594_ECP_30-03-2023_AGROPECUARIA FLAMBOYAN_10.pdf',
 5: '75_ECP_28-03-2023_AGUILERA TARADELLES NELSON MARIANO _5.pdf',
 6: '75_ECP_28-03-2023_AGUILERA TARADELLES NELSON MARIANO _6.pdf',
 13: '765_ECP_31-03-2023_AGROP. CATALA S.R.L._13.pdf',
 14: '765_ECP_31-03-2023_AGROP. CATALA S.R.L._14.pdf',
 15: '765_ECP_31-03-2023_AGROP. CATALA S.R.L._15.pdf',
 16: '765_ECP_31-03-2023_AGROP. CATALA S.R.L._16.pdf',
 8: '8009_ECP_29-03-2023_ESPINOZA AGUILAR DAVID_8.pdf',
 9: '8009_ECP_29-03-2023_ESPINOZA 

In [15]:
# CARGAR LOS PDFS A GOOGLE DRIVE
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "EVAL_MANTENIMIENTO_PREVENTIVO"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(ruta + r'//pdf//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 10697_ECP_28-03-2023_PAZ REA JAIME EDUARDO _4.pdf with mimeType application/pdf
Created file 2446_ECP_16-05-2023_CORDOVA SEJAS ALCIDES _101.pdf with mimeType application/pdf
Created file 3400_ECP_30-03-2023_ESPINOZA AGUILAR JAIME OMAR _11.pdf with mimeType application/pdf
Created file 3469_ECP_22-05-2023_ENRIQUEZ PAICHUCAMA SIMON _106.pdf with mimeType application/pdf
Created file 4171_ECP_19-05-2023_FRITZER CUELLAR SAMIR EDUARDO _104.pdf with mimeType application/pdf
Created file 515_ECP_29-03-2023_AGUILERA OLGA RIVERO VDA. DE _7.pdf with mimeType application/pdf
Created file 556_ECP_16-05-2023_AYALA ESPINOSA WALDO SELUB_100.pdf with mimeType application/pdf
Created file 556_ECP_30-03-2023_AYALA ESPINOSA WALDO SELUB_12.pdf with mimeType application/pdf
Created file 594_ECP_30-03-2023_AGROPECUARIA FLAMBOYAN_10.pdf with mimeType application/pdf
Created file 75_ECP_28-03-2023_AGUILERA TARADELLES NELSON MARIANO _5.pdf with mimeType application/pdf
Created file 75_ECP_28-03-20

In [16]:
dic_urls

{4: 'https://drive.google.com/file/d/17EN7o9XlI_B0MMxgF6A4ysxM8Xb3qFbE/view?usp=drivesdk',
 101: 'https://drive.google.com/file/d/171V_Vyj64mwClHTegfnuEcNnUKaRBMzH/view?usp=drivesdk',
 11: 'https://drive.google.com/file/d/1hV4HtR8qkWnFCgGxgGo8WeJcbBACM9Km/view?usp=drivesdk',
 106: 'https://drive.google.com/file/d/1VKmPKTtlxTgg6OKFNtf2BkFfTLzxjOPO/view?usp=drivesdk',
 104: 'https://drive.google.com/file/d/1kfY8z4KAMp3DJlBgcaMXyY4ALCqSsQyb/view?usp=drivesdk',
 7: 'https://drive.google.com/file/d/1QbxaV65PdH6CrRXkTgS85HWxAnimPf5R/view?usp=drivesdk',
 100: 'https://drive.google.com/file/d/1h31FtHkdLEym5l5ay7quu4KWRDqYRAQJ/view?usp=drivesdk',
 12: 'https://drive.google.com/file/d/1FZAipYso_zgkrXZgfbUBBJuL2amRHplk/view?usp=drivesdk',
 10: 'https://drive.google.com/file/d/10hwJHTow5qFwQpXOId-7gJZe2ISME-e8/view?usp=drivesdk',
 5: 'https://drive.google.com/file/d/1WPi8uKpg-YGPcaJxNKjhhagFk7Y_fRKU/view?usp=drivesdk',
 6: 'https://drive.google.com/file/d/1XnUrq5FxqjIIdXjL6yu2YSusac4fePcd/view?usp

In [17]:
for key in dic_urls:
    query = {'query': 'update dataset_315614 set link_reporte = \'{link}\' where id = {cod}'.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32920/sql', query)
    print(key)
print('end')

4
101
11
106
104
7
100
12
10
5
6
13
14
15
16
8
9
17
end
